In [1]:
from sklearn.model_selection import train_test_split

from transformers import T5Tokenizer, T5ForConditionalGeneration  

import json
from torch.optim import AdamW
import pandas as pd
import torch
torch.set_float32_matmul_precision('high')
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.nn.utils.rnn import pad_sequence
# from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler

pl.seed_everything(100)
import warnings
warnings.filterwarnings("ignore")

d:\HocTap\ChatBot\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Seed set to 100


In [2]:
# mostly pl is used while doing complex model training
import pytorch_lightning as pl
print(pl.__version__)

2.5.1.post0


In [3]:
# argparse makes it easier to write user friendly command line interfaces
import argparse
# package for faster file name matching
import glob
# makiing directories for data 
import os
# reading json files as the data is present in json files
import json
# time module for calculating the model runtime
import time
# Allows writing status messages to a file
import logging
# generate random float numbers uniformly
import random
# regex module for text 
import re
# module provides various functions which work on 
# iterators too produce complex iterators
from itertools import chain
from string import punctuation

# pandas for data manipulation
import pandas as pd
# numpy for array operations
import numpy as np
# PyTorch
import torch
# provides various classes representing file system paths
# with appropriate semantics
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl

# splitting the data 
from sklearn.model_selection import train_test_split
# ANSII color formatting for ouput in terminal
from termcolor import colored
# wrapping paragraphs into string
import textwrap

# model checkpoints in pretrained model
from pytorch_lightning.callbacks import ModelCheckpoint

'''
optimizer - AdamW
T5 Conditional Generator in which we'll give conditions
T5 tokenizer because it is fast
training the model without a learning rate
'''
from transformers import (
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

In [4]:
# check the version provided by Lightning
import pytorch_lightning as pl
print(pl.__version__)

2.5.1.post0


In [5]:
from datasets import load_dataset

# Load dataset
dataset = load_dataset('csv', data_files='../dataset/QA_data.csv')
print(dataset)
dataset = dataset['train']

DatasetDict({
    train: Dataset({
        features: ['Question', 'Context', 'Answer', 'Answer_Start', 'Answer_End'],
        num_rows: 197379
    })
})


In [6]:
# len of each file
len(dataset)

197379

In [7]:
import pandas as pd
dataset = pd.DataFrame(dataset)

In [8]:
dataset.shape

(197379, 5)

In [9]:
len(dataset['Question'].unique())

171723

In [10]:
len(dataset['Context'].unique())

196625

In [11]:
sample_question = dataset.iloc[243]
sample_question

Question        dạ cho em hỏi em bị ngứa cổ họng ho và sổ mũi ...
Context         Có thể là bạn bị viêm mũi họng dị ứng có kèm b...
Answer                                                       None
Answer_Start                                                   -1
Answer_End                                                     -1
Name: 243, dtype: object

In [12]:
# Using textcolor to visualize the answer within the context
from termcolor import colored
def color_answer(example):
  answer_start, answer_end = example["Answer_Start"],example["Answer_End"]
  context = example['Context']

  return  colored(context[:answer_start], "white") + \
    colored(context[answer_start:answer_end + 1], "green") + \
    colored(context[answer_end+1:], "white")

In [13]:
print(dataset.iloc[1000])
print()
print("Answer: ")
for wrap in textwrap.wrap(color_answer(dataset.iloc[1000]), width = 100):
  print(wrap)

Question        Dạ bác sĩ cho em hỏi với ạ. Ông , bà của em đề...
Context         Chào bạn, ông bà bạn cao tuổi, bà có bệnh nền ...
Answer          Chào bạn, ông bà bạn cao tuổi, bà có bệnh nền ...
Answer_Start                                                    0
Answer_End                                                    247
Name: 1000, dtype: object

Answer: 
Chào bạn, ông bà bạn cao tuổi, bà có bệnh nền , cả 2 ông bà đều nên tiêm nhé. Sau tiêm
có tác dụng phụ tuỳ ông bà sốt từ 38,5 độ C uống thuốc hạ sốt, uống nhiều nước, uống thêm nước cam,
chanh, dừa để tăng sức đề kháng và hạn chế mất nước khi sốt. Ăn uống nhẹ nhàng đồ dễ tiêu
thôi nhé. Dạ , dạ e cám ơn chị bác sĩ rất nhiều ạ


Tokenization

In [ ]:
# using the base T5 model having 222M params
MODEL_NAME ='t5-base'

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)

To create dataset

In [ ]:
class BioQADataset(Dataset):
  def __init__(
      self,
      data:pd.DataFrame,
      tokenizer:T5Tokenizer,
      source_max_token_len: int = 396,
      target_max_token_len: int = 32,

      ):
    
    self.data =  data
    self.tokenizer =  tokenizer
    self.source_max_token_len =  source_max_token_len
    self.target_max_token_len =  target_max_token_len


  def __len__(self):
    return len(self.data)

  def __getitem__(self, index: int):
    data_row = self.data.iloc[index]

    source_encoding = tokenizer(
      data_row['question'],
      data_row['context'],
      max_length=self.source_max_token_len,
      padding='max_length',
      truncation="only_second",
      return_attention_mask=True,
      add_special_tokens=True,
      return_tensors="pt"
      )
    
    target_encoding = tokenizer(
      data_row['answer_text'],
      max_length=self.target_max_token_len,
      padding='max_length',
      truncation=True,
      return_attention_mask=True,
      add_special_tokens=True,
      return_tensors="pt"
      )
    
    labels = target_encoding['input_ids']
    labels[labels==0] = -100

    return dict(
        question=data_row['question'],
        context=data_row['context'],
        answer_text=data_row['answer_text'],
        input_ids=source_encoding["input_ids"].flatten(),
        attention_mask=source_encoding['attention_mask'].flatten(),
        labels=labels.flatten()
    )

In [ ]:
sample_dataset = BioQADataset(df, tokenizer)

In [ ]:
for data in sample_dataset:
  print("Question: ", data['question'])
  print("Answer text: ", data['answer_text'])
  print("Input_ids: ", data['input_ids'][:10])
  print("Labels: ", data['labels'][:10])
  break

Splitting into train and validation sets

In [ ]:
train_df, val_df = train_test_split(df, test_size=0.05)

In [ ]:
train_df.shape,  val_df.shape

Create pytorch lightning datamodule


In [ ]:
class BioDataModule(pl.LightningDataModule):
  def __init__(
      self,
      train_df: pd.DataFrame,
      test_df: pd.DataFrame,
      tokenizer:T5Tokenizer,
      batch_size: int = 8,
      source_max_token_len: int = 396,
      target_max_token_len: int = 32,
      ):
    super().__init__()
    self.train_df = train_df
    self.test_df = test_df
    self.tokenizer = tokenizer
    self.batch_size = batch_size
    self.source_max_token_len = source_max_token_len
    self.target_max_token_len = target_max_token_len

  def setup(self, stage=None):
    self.train_dataset = BioQADataset(
        self.train_df,
        self.tokenizer,
        self.source_max_token_len,
        self.target_max_token_len
        )

    self.test_dataset = BioQADataset(
    self.test_df,
    self.tokenizer,
    self.source_max_token_len,
    self.target_max_token_len
    )
 
  def train_dataloader(self):
    return DataLoader(
        self.train_dataset,
        batch_size=self.batch_size,
        shuffle=True,
        num_workers=0
        )
  def val_dataloader(self):
    return DataLoader(
        self.test_dataset,
        batch_size=self.batch_size,
        num_workers=0
        )

  def test_dataloader(self):
    return DataLoader(
        self.test_dataset,
        batch_size=1,
        num_workers=0
        )

In [ ]:
BATCH_SIZE = 8
N_EPOCHS = 1

data_module = BioDataModule(train_df, val_df, tokenizer, batch_size=BATCH_SIZE)
data_module.setup()

Loading and finetuning the T5-base model

In [ ]:
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME, return_dict = True)

In [ ]:

model.to(device)

In [ ]:
model.config

In [ ]:
# To check the translation from English to German built-in task 

input_ids_translated = tokenizer(
    "translate English to German : Oppertunity did not knock until I built a door",
    return_tensors = 'pt'
).input_ids.to(device)

generated_ids = model.generate(input_ids = input_ids_translated)
generated_ids

In [ ]:
pred_translated = [
         tokenizer.decode(gen_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
         for gen_id in generated_ids
]

In [ ]:
pred_translated

In [ ]:
"".join(pred_translated)

In [ ]:
# To check the summarization built-in task

text = """summarize : Musk was born to a Canadian mother and South African father and raised in Pretoria, South Africa. 
He briefly attended the University of Pretoria before moving to Canada aged 17 to attend Queen's University. 
He transferred to the University of Pennsylvania two years later, where he received bachelor's degrees in economics and physics. 
He moved to California in 1995 to attend Stanford University but decided instead to pursue a business career, 
co-founding the web software company Zip2 with his brother Kimbal. The startup was acquired by Compaq for $307 million in 1999. 
Musk co-founded online bank X.com that same year, which merged with Confinity in 2000 to form PayPal. 
The company was bought by eBay in 2002 for $1.5 billion. In 2002, Musk founded SpaceX, an aerospace manufacturer and space transport 
services company, of which he is CEO and CTO. In 2004, he joined electric vehicle manufacturer Tesla Motors, Inc. (now Tesla, Inc.) 
as chairman and product architect, becoming its CEO in 2008. In 2006, he helped create SolarCity, a solar energy services company that 
was later acquired by Tesla and became Tesla Energy. In 2015, he co-founded OpenAI, a nonprofit research company that promotes friendly 
artificial intelligence. In 2016, he co-founded Neuralink, a neurotechnology company focused on developing brain–computer interfaces, 
and founded The Boring Company, a tunnel construction company. Musk has proposed the Hyperloop, a high-speed vactrain transportation system."""

In [ ]:
input_ids_summary = tokenizer(
    text,
    return_tensors = 'pt'
).input_ids.to(device)

generated_ids_summary = model.generate(input_ids = input_ids_summary)
generated_ids_summary

In [ ]:
pred_summary = [
         tokenizer.decode(gen_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
         for gen_id in generated_ids
]

In [ ]:
" ".join(pred_summary)

In [ ]:
# Model config

model.config

In [ ]:
output = model(
    input_ids = encoding['input_ids'],
    attention_mask = encoding['attention_mask'],
    labels = labels
)

In [ ]:
output.logits.shape

In [ ]:
output.loss

Building the PyTorch lightning module using T5ForConditionalGeneration model

In [ ]:
class BioQAModel(pl.LightningModule):
  def __init__(self):
    super().__init__()
    self.model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME, return_dict=True)


  def forward(self, input_ids, attention_mask, labels=None):
    output = self.model(
        input_ids, 
        attention_mask=attention_mask,
        labels=labels)

    return output.loss, output.logits

  def training_step(self, batch, batch_idx):
    input_ids = batch['input_ids']
    attention_mask=batch['attention_mask']
    labels = batch['labels']
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("train_loss", loss, prog_bar=True, logger=True)
    return {"loss": loss, "predictions":outputs, "labels": labels}

  def validation_step(self, batch, batch_idx):
    input_ids = batch['input_ids']
    attention_mask=batch['attention_mask']
    labels = batch['labels']
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("val_loss", loss, prog_bar=True, logger=True)
    return loss

  def test_step(self, batch, batch_idx):
    input_ids = batch['input_ids']
    attention_mask=batch['attention_mask']
    labels = batch['labels']
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("test_loss", loss, prog_bar=True, logger=True)
    return loss

  def configure_optimizers(self):

    optimizer = AdamW(self.parameters(), lr=0.0001)
    return optimizer

In [ ]:
model = BioQAModel() 

Using trainer from pytorch lightning to finetune model using our dataset

In [ ]:
# To record the best performing model using checkpoint

checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints",
    filename="best-checkpoint",
    save_top_k=1,
    verbose=True,
    monitor="val_loss",
    mode="min"
)


In [ ]:
from pytorch_lightning.callbacks import ProgressBar

# Khởi tạo ProgressBar callback
progress_bar_callback = ProgressBar()
trainer = pl.Trainer(
    callbacks=checkpoint_callback,
    max_epochs=N_EPOCHS,
    accelerator='gpu',
    devices = 1
)

In [ ]:
trainer.fit(model, data_module)

Predictions

In [ ]:
trained_model = BioQAModel.load_from_checkpoint("checkpoints/best-checkpoint.ckpt")
trained_model.freeze() # 

Generate answers for the questions in the validation set

In [ ]:
def generate_answer(question):
  source_encoding=tokenizer(
      question["question"],
      question['context'],
      max_length = 396,
      padding="max_length",
      truncation="only_second",
      return_attention_mask=True,
      add_special_tokens=True,
      return_tensors="pt"

  ).to(device)

  generated_ids = trained_model.model.generate(
      input_ids=source_encoding["input_ids"],
      attention_mask=source_encoding["attention_mask"],
      num_beams=1,  # greedy search
      max_length=80,
      repetition_penalty=2.5,
      early_stopping=True,
      use_cache=True)
  
  preds = [
          tokenizer.decode(generated_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
          for generated_id in generated_ids
  ]

  return "".join(preds)

In [ ]:
sample_question = val_df.iloc[20]

In [ ]:
sample_question["question"]

In [ ]:
sample_question["answer_text"]  # Label Answer

In [ ]:
generate_answer(sample_question)  # Predicted answer